In [1]:
# repo root
%cd /work

/work


In [2]:
!ls 

LICENSE    adapters  examples  paper	       resources  src
README.md  docker    out       pyproject.toml  scripts	  tests


In [3]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

In [4]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

In [5]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [6]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [337]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/sample_cards/*
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [14]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "note",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [15]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "hard",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [10]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [11]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
 [100%] Downloaded 33730584 bytes...
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [338]:
# Fig2 run
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours,context_swap \
  --k-claims 50 \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours,context_swap
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 10
[run_fig2] (1/10) HNSC ours gate=hard tau=0.20
[run_fig2] (2/10) HNSC context_swap gate=hard tau=0.20
[run_fig2] (3/10) HNSC ours gate=hard tau=0.40
[run_fig2] (4/10) HNSC context_swap gate=hard tau=0.40
[run_fig2] (5/10) HNSC ours gate=hard tau=0.60
[run_fig2] (6/10) HNSC context_swap gate=hard tau=0.60
[run_fig2] (7/10) HNSC ours gate=hard tau=0.80
[run_fig2] (8/10) HNSC context_swap gate=hard tau=0.80
[run_fig2] (9/10) HNSC ours gate=hard tau=0.90
[run_fig2] (10/10) HNSC context_swap gate=hard tau=0.90
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [339]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC

context_swap  ours


In [340]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv		    modules.tsv		  sample_card.resolved.json
claims.proposed.tsv	    report.jsonl	  sample_card.tau.json
distilled.tsv		    report.md		  term_gene_edges.tsv
distilled.with_modules.tsv  risk_coverage.tsv	  term_modules.tsv
evidence.normalized.tsv     run_meta.json
manifest.json		    run_meta.runner.json


In [341]:
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json


{
  "tool": "llm-pathway-curator",
  "cmd": "run",
  "run_id": "1769210686_66187f0376",
  "status": "ok",
  "started_epoch": 1769210686.5640576,
  "config": {
    "evidence_table": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
    "sample_card": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json",
    "outdir": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80",
    "force": true,
    "seed": 42,
    "run_meta_name": "run_meta.json",
    "tau": 0.8,
    "k_claims": 50
  },
  "env": {
    "python": "3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:16:53) [GCC 14.3.0]",
    "platform": "Linux-6.12.54-linuxkit-aarch64-with-glibc2.39",
    "pandas": "2.3.3"
  },
  "inputs": {
    "evidence_table": {
      "path": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
      "size_bytes": 12916,
      "mtime_epoch": 1769181104.7039697
    },
    "sample_c

In [342]:
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"

import pandas as pd
p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")
print("shape", df.shape)
print(df["status"].value_counts(dropna=False))
for c in ["abstain_reason","fail_reason","context_status","context_evaluated","context_method","audit_notes"]:
    if c in df.columns:
        print("\n==", c, "==")
        print(df[c].fillna("").value_counts().head(20))

shape (50, 76)
status
ABSTAIN    50
Name: count, dtype: int64

== abstain_reason ==
abstain_reason
context_nonspecific    43
unstable                7
Name: count, dtype: int64

== fail_reason ==
fail_reason
    50
Name: count, dtype: int64

== context_status ==
context_status
WARN    43
PASS     7
Name: count, dtype: int64

== context_evaluated ==
context_evaluated
True     43
False     7
Name: count, dtype: int64

== context_method ==
context_method
proxy    43
none      7
Name: count, dtype: int64

== audit_notes ==
audit_notes
context_warn(hard): proxy_context: context_score=0 < pass_min=1    43
survival[module]=0.766 < tau=0.80                                   3
survival[module]=0.750 < tau=0.80                                   2
survival[module]=0.797 < tau=0.80                                   1
survival[module]=0.672 < tau=0.80                                   1
Name: count, dtype: int64


In [343]:
import json, pandas as pd

p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")

def has_keys(s, keys):
    try:
        o = json.loads(str(s))
        return {k: (k in o) for k in keys}
    except Exception:
        return None

keys = ["context_evaluated","context_method","context_status","context_reason","context_confidence","context_notes"]
sample = df[df["abstain_reason"]=="context_missing"].head(5)

for i, r in sample.iterrows():
    ck = has_keys(r.get("claim_json",""), keys)
    print("row", i, "claim_json_has", ck)
    # ついでに中身の該当フィールドだけ見たいなら：
    try:
        o = json.loads(r["claim_json"])
        print({k:o.get(k,None) for k in keys})
    except Exception as e:
        print("json load error:", e)
    print("---")


In [344]:
import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("\n".join(df.columns))

claim_id
entity
direction
context_keys
term_uid
source
term_id
term_name
module_id
module_missing
module_reason
module_prefix_effective
gene_ids
gene_ids_suggest
gene_ids_n_total
gene_ids_n_in_claim
term_ids
gene_set_hash
context_score
context_evaluated
eligible
term_survival
keep_term
keep_reason
claim_json
preselect_tau_gate
context_score_proxy
select_context_mode
context_tiebreak
select_notes
select_diag_n_total
select_diag_n_eligible
select_diag_n_ineligible
select_diag_n_picked
select_diag_blocked_by_module_cap
select_diag_relaxed
select_diag_relax_passes
select_diag_k
select_diag_max_per_module
claim_mode
llm_notes
context_method
context_status
context_reason
context_notes
context_confidence
context_gate_mode
gene_ids_excel
term_ids_excel
context_review_mode
status
link_ok
stability_ok
contradiction_ok
stress_ok
stress_evaluated
abstain_reason
fail_reason
audit_notes
tau_used
term_survival_agg
stability_scope
module_id_effective
gene_set_hash_effective
term_ids_set_hash
evidence_

In [350]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20
report   True 109943
audit    True 103211
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 906
card     True 547
meta     True 6081

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 63)
[report] decisions: {'ABSTAIN': 50}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id method 

In [351]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20
report   True 110343
audit    True 103211
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 906
card     True 630
meta     True 6191

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 63)
[report] decisions: {'ABSTAIN': 50}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id

In [352]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv		    modules.tsv		  sample_card.resolved.json
claims.proposed.tsv	    report.jsonl	  sample_card.tau.json
distilled.tsv		    report.md		  term_gene_edges.tsv
distilled.with_modules.tsv  risk_coverage.tsv	  term_modules.tsv
evidence.normalized.tsv     run_meta.json
manifest.json		    run_meta.runner.json


In [353]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 109656
audit    True 102511
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 906
card     True 547
meta     True 6081

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 63)
[report] decisions: {'ABSTAIN': 50}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id method 

In [354]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80
report   True 110056
audit    True 102511
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 906
card     True 630
meta     True 6192

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 63)
[report] decisions: {'ABSTAIN': 50}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id

In [139]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants stress \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 100 \
  --stress-evidence-dropout-p 0.05 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours,stress
  gate_modes: hard
  taus: 0.80
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
  stress:
    recipe: dropout
    evidence_dropout_p: 0.050
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 2
[run_fig2] (1/2) HNSC ours gate=hard tau=0.80
[run_fig2] (2/2) HNSC stress gate=hard tau=0.80
[run_fig2] WARNING: stress variant ran but pipeline recorded both evidence_dropout_p=0 and contradictory_p=0 (stress may not have been applied). outdir=/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [49]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 98638
audit    True 83640
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 906
card     True 512
meta     True 5883

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 63)
[report] decisions: {'PASS': 43, 'ABSTAIN': 7}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_i

In [270]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80
report   False None
audit    False None
modules  False None
edges    False None
terms    False None
distilled False None
risk     False None
card     False None
meta     False None
[SANITY][ERROR] missing required artifact: report -> /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/report.jsonl


In [282]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*

!LLMPATH_BACKEND="ollama" \
LLMPATH_OLLAMA_HOST="http://ollama:11434" \
LLMPATH_OLLAMA_MODEL="llama3.1:8b" \
LLMPATH_CLAIM_MODE=llm \
python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 50 \
  --context-review-mode llm \
  --force

OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"
!cat $OUT/llm_claims.raw.json
!cat $OUT/llm_claims.meta.json

import json
m=json.load(open("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json"))
print(m["inputs"]["llm"])

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours
  gate_modes: hard
  taus: 0.80
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC ours gate=hard tau=0.80
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
{
  "raw": "{\"error\":{\"message\":\"timed out\",\"retryable\":false,\"type\":\"ollama_error\"}}"
}
{
  "k": 50,
  "seed": 42,
  "top_n": 12,
  "require_gene_set_hash": true,
  "context_keys_resolved": [
    "condition",
    "tissue",
    "perturbation",
    "comparison"
  ],
  "context_proxy_enabled": false,
  "candidates_sha256": "906a3d99d346216f4c3fbe34468eb7ea99a24d785a744dba525287e2e224e0cd",
  "notes": "soft_error: timed out",
  "backend_class": "OllamaBackend",
  "artifact_tag": ""
}
{'claim': {'enabled': True, 'claim_mode_env': 'llm', 'backend_env': 'ollama', 'backend_enabled': True, 'backend_notes': ''}, 'review': {'enabled': True, 'review_mode': 'llm', 

In [276]:
import pandas as pd
df=pd.read_csv("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv", sep="\t")
print([c for c in df.columns if c.startswith("context")])

['context_key', 'context_keys', 'context_score', 'context_review_evaluated', 'context_review_status', 'context_review_reason', 'context_review_notes', 'context_review_confidence', 'context_review_mode', 'context_review_method', 'context_evaluated', 'context_method', 'context_status', 'context_reason', 'context_notes', 'context_confidence', 'context_gate_mode']


In [277]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"

!ls -lh $OUT/{run_meta.json,claims.proposed.tsv,audit_log.tsv,report.jsonl} 2>/dev/null || true
!wc -l  $OUT/claims.proposed.tsv  $OUT/audit_log.tsv  $OUT/report.jsonl 2>/dev/null || true

import json, os
p=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")+"/run_meta.json"
m=json.load(open(p))
print("step:", m.get("step"), "status:", m.get("status"))
print("llm:", m.get("inputs",{}).get("llm",{}))
print("claims:", m.get("inputs",{}).get("claims",{}))

   13 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv
   13 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv
   12 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl
   38 total
step: report_jsonl status: ok
llm: {'claim_mode_env': 'llm', 'backend_env': 'ollama', 'backend_enabled': True, 'backend_notes': '', 'backend_config': {'backend': 'ollama', 'host': 'http://ollama:11434', 'model_name': 'llama3.1:8b', 'temperature': 0.0, 'timeout': 120.0}}
claims: {'context_gate_review_compat': {'compat_action': 'none', 'gate_mode': 'hard', 'review_mode': 'llm'}, 'k_source': 'cfg', 'k_cfg': 100, 'k_env': '', 'k_card': '', 'k_effective': 100, 'k_env_raw': '', 'n_proposed_rows': 12, 'n_proposed_cols': 40, 'claim_payload_col': 'claim_json', 'claim_json_present': True, 'context_gate_mode': 'hard', 'context_review_mode': 'llm', 'context_review_runtime': {'evaluated': True, 'mode': 'llm', '

In [283]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 109319
audit    True 101187
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 906
card     True 509
meta     True 6196

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 63)
[report] decisions: {'ABSTAIN': 50}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id method 

In [273]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"

import pandas as pd, os, json
out=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")
p=f"{out}/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
cols=[c for c in df.columns if c.startswith("context_review_")]
print("context_review_* cols:", cols)
print(df[cols].head(3) if cols else "(none)")

import pandas as pd, os, json
out=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")
df=pd.read_csv(f"{out}/claims.proposed.tsv", sep="\t")
cj=df.loc[0,"claim_json"]
obj=json.loads(cj)
keys=[k for k in obj.keys() if k.startswith("context_")]
print("claim_json context_* keys:", keys)
print({k: obj.get(k) for k in keys})

context_review_* cols: ['context_review_evaluated', 'context_review_status', 'context_review_reason', 'context_review_notes', 'context_review_confidence', 'context_review_mode', 'context_review_method']
   context_review_evaluated context_review_status context_review_reason  \
0                      True                  PASS   TP53_mut_vs_TP53_wt   
1                      True                  PASS   TP53_mut_vs_TP53_wt   
2                      True                  PASS   TP53_mut_vs_TP53_wt   

                                context_review_notes  \
0  Comparison is between TP53 mutant and wildtype...   
1  Comparison of TP53 mutant vs wildtype in HNSC ...   
2  Comparison of TP53 mutant vs wildtype in HNSC ...   

   context_review_confidence context_review_mode context_review_method  
0                        0.8                 llm                   llm  
1                        1.0                 llm                   llm  
2                        1.0                 llm    

In [274]:
import json, pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
row=df.iloc[0]
cj=json.loads(row["claim_json"])
print("context_keys:", cj.get("context_keys"))

context_keys: ['condition']


In [250]:
import pandas as pd
p="paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("cols:", [c for c in df.columns if "claim_json" in c])
print("has_claim_json:", "claim_json" in df.columns)
print("has_claim_json_str:", "claim_json_str" in df.columns)

cols: ['claim_json']
has_claim_json: True
has_claim_json_str: False


In [251]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"
!ls -lh "$OUT" | sed -n '1,200p'

for f in ["distilled.tsv", "distilled_with_modules.tsv modules.tsv", "evidence.normalized.tsv", "evidence.tsv",]:
  ![ -f "$OUT/$f" ] && echo "== $f ==" && wc -l "$OUT/$f" && head -n 3 "$OUT/$f"



total 392K
-rw-r--r-- 1 root root  18K Jan 23 19:35 audit_log.tsv
-rw-r--r-- 1 root root  16K Jan 23 19:35 claims.proposed.tsv
-rw-r--r-- 1 root root 3.0K Jan 23 19:35 context_review_cache.jsonl
-rw-r--r-- 1 root root  40K Jan 23 19:35 distilled.tsv
-rw-r--r-- 1 root root  40K Jan 23 19:29 distilled.with_modules.tsv
-rw-r--r-- 1 root root  15K Jan 23 19:29 evidence.normalized.tsv
-rw-r--r-- 1 root root  43K Jan 23 19:33 llm_claims.candidates.json
-rw-r--r-- 1 root root  379 Jan 23 19:33 llm_claims.meta.json
-rw-r--r-- 1 root root 3.2K Jan 23 19:33 llm_claims.prompt.json
-rw-r--r-- 1 root root 2.4K Jan 23 19:33 llm_claims.raw.json
-rw-r--r-- 1 root root 2.3K Jan 23 19:35 manifest.json
-rw-r--r-- 1 root root  14K Jan 23 19:29 modules.tsv
-rw-r--r-- 1 root root  21K Jan 23 19:35 report.jsonl
-rw-r--r-- 1 root root  42K Jan 23 19:35 report.md
-rw-r--r-- 1 root root 6.0K Jan 23 19:35 run_meta.json
-rw-r--r-- 1 root root  654 Jan 23 19:29 run_meta.runner.json
-rw-r--r-- 1 root root  510 Jan 

In [252]:
import pandas as pd
p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/distilled.tsv"
df = pd.read_csv(p, sep="\t")
print(df.shape)
for c in ["keep_term","eligible","term_survival","keep_reason"]:
    if c in df.columns:
        print(c, df[c].value_counts(dropna=False).head(10))

(50, 54)
keep_term keep_term
True    50
Name: count, dtype: int64
term_survival term_survival
0.984375    10
1.000000     7
0.968750     7
0.953125     5
0.921875     4
0.906250     3
0.937500     3
0.750000     2
0.843750     2
0.875000     1
Name: count, dtype: int64
keep_reason keep_reason
ok    50
Name: count, dtype: int64


In [253]:
import pandas as pd, json
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
print("rows:", df.shape[0])
print("context_confidence unique:", df.get("context_confidence").unique() if "context_confidence" in df.columns else None)
print("context_review_confidence unique:", df.get("context_review_confidence").unique() if "context_review_confidence" in df.columns else None)


rows: 12
context_confidence unique: [0.8 0.9]
context_review_confidence unique: [0.8 0.9]


In [254]:
# context未評価でABSTAINになっているか
!grep -n '"decision_status": "ABSTAIN"' \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl | head

# audit_notes に context_missing(hard) が残っているか
!grep -n "context_missing(hard)" \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl | head


In [255]:
# 1) DNS/ネットワーク
!getent hosts ollama || ping -c 1 ollama

# 2) Ollama API 生存確認
!curl -sS http://ollama:11434/api/tags | head

# 3) 推論が通るか（最小）
!curl -sS http://ollama:11434/api/generate \
  -H 'Content-Type: application/json' \
  -d '{"model":"llama3.1:8b","prompt":"Say OK","stream":false}' | head


172.18.0.2      ollama
{"models":[{"name":"llama3.1:8b","model":"llama3.1:8b","modified_at":"2026-01-01T16:13:01.415171001Z","size":4920753328,"digest":"46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"Q4_K_M"}}]}{"model":"llama3.1:8b","created_at":"2026-01-23T19:36:05.304008004Z","response":"OK!","done":true,"done_reason":"stop","context":[128006,882,128007,271,46864,10619,128009,128006,78191,128007,271,4012,0],"total_duration":1594312750,"load_duration":109052958,"prompt_eval_count":12,"prompt_eval_duration":1297708792,"eval_count":3,"eval_duration":179630708}

In [ ]:
import os, json, requests
host = os.environ.get("LLMPATH_OLLAMA_HOST","http://ollama:11434")
model = os.environ.get("LLMPATH_OLLAMA_MODEL","llama3.1:8b")
payload = {"model": model, "prompt": "Say OK", "stream": False}
r = requests.post(f"{host}/api/generate", json=payload, timeout=120)
print("status", r.status_code)
print("text", r.text[:200])
print("json", r.json())

In [ ]:
import inspect
import llm_pathway_curator.backends as b
print("backends.py:", b.__file__)
print("--- OllamaBackend.generate source ---")
print(inspect.getsource(b.OllamaBackend.generate))

In [ ]:
from llm_pathway_curator.backends import get_backend_from_env
b = get_backend_from_env(seed=42)
print("backend:", type(b).__name__, getattr(b, "host", None), getattr(b, "model_name", None))

s = b.generate('{"ping": "pong"}', json_mode=True)
print("returned:", s[:200])

In [ ]:
!grep -RIn "backend_call_failed" -n src/llm_pathway_curator
!grep -RIn "llm_claims.raw.json" -n src/llm_pathway_curator
!grep -RIn "llm_claims.meta.json" -n src/llm_pathway_curator


In [ ]:
import json
from llm_pathway_curator.backends import get_backend_from_env

b = get_backend_from_env(seed=42)
print("backend:", type(b).__name__, b.host, b.model_name)

# 1) json_mode で “スキーマっぽい” JSONを返させる
prompt = '{"claims": [{"claim_id":"c1","entity":"X","direction":"up","evidence_refs":["m1"],"context":{"disease":"HNSC"} }]}'
out = b.generate(prompt, json_mode=True)
print("raw_out_head:", out[:200])

# 2) ちゃんとJSONとして読めるか
obj = json.loads(out)
print("top_keys:", list(obj.keys())[:10])

In [77]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [142]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,shuffled_context \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  variants: ours,shuffled_context
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 70
[run_fig2] (1/70) HNSC ours gate=hard tau=0.20
[run_fig2] (2/70) HNSC shuffled_context gate=hard tau=0.20
[run_fig2] (3/70) HNSC ours gate=hard tau=0.40
[run_fig2] (4/70) HNSC shuffled_context gate=hard tau=0.40
[run_fig2] (5/70) HNSC ours gate=hard tau=0.60
[run_fig2] (6/70) HNSC shuffled_context gate=hard tau=0.60
[run_fig2] (7/70) HNSC ours gate=hard tau=0.80
[run_fig2] (8/70) HNSC shuffled_context gate=hard tau=0.80
[run_fig2] (9/70) HNSC ours gate=hard tau=0.90
[run_fig2] (10/70) HNSC shuffled_context gate=hard tau=0.90
[run_fig2] (11/70) LUAD ours gate=hard tau=0.20
[run_fig2] (12/70) LUAD shuffled_context gate=hard tau=0.20
[run_fig2] (13/70) LUAD ours gate=hard tau=0.40
[run_fig2] (14/70) LUAD shuffled_c

In [143]:
!python paper/scripts/fig2_make_labels_template.py \
  --root paper/source_data/PANCAN_TP53_v1/out \
  --out  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --per-run 20 \
  --prefer-status PASS \
  --with-hints

[OK] wrote: paper/source_data/PANCAN_TP53_v1/labels.tsv rows=150
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [141]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --benchmark-id PANCAN_TP53_v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv rows=70
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [144]:
!python paper/scripts/bench_fig2.py \
  --reports paper/source_data/PANCAN_TP53_v1/out \
  --labels  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --out     /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --taus    0:1:0.05 \
  --rater-id R1


Traceback (most recent call last):
  File "/work/paper/scripts/bench_fig2.py", line 315, in <module>
    main()
  File "/work/paper/scripts/bench_fig2.py", line 303, in main
    reports_df = load_reports(report_paths)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/work/paper/scripts/bench_fig2.py", line 123, in load_reports
    raise ValueError(
ValueError: Missing metrics.survival in paper/source_data/PANCAN_TP53_v1/out/BRCA/ours/gate_hard/tau_0.20/report.jsonl line 1 (claim_id=c_a1ed09ef8099)


In [104]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --out paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_proxy.pdf \
  --ycol risk_proxy \
  --benchmark-id PANCAN_TP53_v1 \
  --cols 4 \
  --fontsize 16 \
  --title "Risk–coverage curves across cancers"
